In [1]:
using Revise

if !isdefined(Main, :SimStepInfo)
    struct SimStepInfo
        user_ind::Int64
        num_clusters::Int64
        recomended_movie::Int64
        actual_rating::Float64
        adjusted_rating::Float64
        predicted_rating::Float64
        counts::Vector{Int64}
    end

    
end

if !isdefined(Main, :TestState)
    struct TestState
        user_ind
        movie_queues
    end
end

struct MDP
    γ  # discount factor
    𝒮  # state space
    𝒜  # action space
    T  # transition function
    R  # reward function
    TR # sample transition and reward
end

struct PickingState
    step_ind
    user_ind
end

struct Environment
    movie_queues
    vector_clusters
end



# Policy that maintains belief
struct DirichletPolicy #<: Policy 
    belief 
end

SimStepInfo() = SimStepInfo(1, 7, 9, 0, 0, 0, ones(7))
PickingState()=PickingState(0)



MDP()=MDP(0,0,0,0,0,0)










MDP

In [1]:
function simulate_episode(mdp, policy,environment,n_steps)
    S_0=PickingState()
    for i in 1:n_steps
        a=policy()
    end
end

mdp=MDP()
policy=DirichletPolicy(ones(7))
env=Environment(1,0)
n_steps=100



UndefVarError: UndefVarError: `MDP` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [2]:
struct MDP
    γ  # discount factor
    𝒮  # state space
    𝒜  # action space
    T  # transition function
    R  # reward function
    TR # sample transition and reward
end

mutable struct Environment
    available_movies
    movie_reviews
end


In [ ]:
num_movies=20
num_users=10
env=Environment(Set(1:20),rand(1:5,num_users,num_movies))
reward(env,user,movie)=env.movie_reviews[user,movie]
#function update!(env, user, movie)
#    env.available_movies = setdiff(env.available_movies, Set([movie]))
#end 
update!(env,1,1)
new_env(env,movie)=Environment(setdiff(env.available_movies, Set([movie])),env.movie_reviews)
env=new_env(env,1)

policy(env::Environment, user::Int, movie::Int) = rand(env.available_movies)
policy(env,1,1)

#simulate_episode
#keep track of 
#1. user
#2. movie
#3. reward
#4. action
#5. state
#6. counts
#7. predicted rating
#8. actual rating
all_tracking=[]
for i in 1:10
    tracking=Dict{String,Any}()
    a=policy(env,1,1)
    r=reward(env,1,a)
    env=new_env(env,a)
    println("a=$(a),r=$(r)")
    tracking["i"]=i
    tracking["user"]=1
    tracking["movie"]=a
    tracking["reward"]=r
    tracking["action"]=a
    #tracking["state"]=env
    all_tracking=push!(all_tracking,tracking)
end
all_tracking
all_rewards=[x["reward"] for x in all_tracking]

a=16,r=3
a=19,r=3
a=7,r=5
a=8,r=5
a=5,r=3
a=17,r=3
a=18,r=3
a=10,r=3
a=4,r=3
a=12,r=4


10-element Vector{Int64}:
 3
 3
 5
 5
 3
 3
 3
 3
 3
 4

now load movies and try a random movie policy to see if it works

In [1]:
include("io.jl")
include("types.jl")
include("utils.jl")
include("preprocessing.jl")
include("svd_model.jl")
using Clustering
using LinearAlgebra
using DataFrames
using Plots

In [2]:

mutable struct Environment
    available_movies
    movie_reviews
end


In [3]:

println("Starting up...")

path="data/ml-32m/"

#args = parse_args()
args=Dict("k"=>4, "num_clusters"=>7, "train_ratio"=>0.8, "min_ratings_per_user"=>1000, "min_ratings_per_movie"=>5000)
config = ClusteringConfig(k=args["k"], num_clusters=args["num_clusters"], train_ratio=args["train_ratio"], min_ratings_per_user=args["min_ratings_per_user"], min_ratings_per_movie=args["min_ratings_per_movie"])
config = ClusteringConfig(k=args["k"], num_clusters=args["num_clusters"], train_ratio=args["train_ratio"], min_ratings_per_user=args["min_ratings_per_user"], min_ratings_per_movie=args["min_ratings_per_movie"])
validate_config(config)

df = load_ratings(path*"ratings.csv")
ratings_data=preprocess_ratings(df, config.min_ratings_per_user, config.min_ratings_per_movie)

println("Loaded $(size(df, 1)) ratings")
println("Loaded $(length(ratings_data.user_map)) users")
(train_users, test_users) = split_users(Set(keys(ratings_data.user_map)), config.train_ratio)

dataset_train = get_data_for_user_subset(ratings_data, Set(train_users))
dataset_test= get_data_for_user_subset(ratings_data, Set(test_users))

#U, S, V = svd(dataset_train.imputed_matrix)
svd_result = create_svd_model(dataset_train.imputed_matrix)
truncated_svd_result = truncate_svd(svd_result, config.k)
println("Truncated SVD to $(config.k) dimensions")

println("Split $(length(train_users)) train users and $(length(test_users)) test users")
println("train_users: $(train_users)")

S=[svd_result.S[i, i] for i in 1:size(svd_result.S, 1)]

println("dimensions of matrices: U=$(size(svd_result.U)), S=$(size(svd_result.S)), V=$(size(svd_result.V))")
p1=plot(cumsum(S) ./ sum(S), legend=false, xlabel="Number of singular values", ylabel="Fraction of total variance", title="Fraction of total variance vs. number of singular values")
savefig(p1, "cumulative_variance_plot.png")
p2=plot(S./sum(S), xlim=[0,10],legend=false, xlabel="Number of singular values", ylabel="Fraction of total variance", title="Fraction of total variance vs. number of singular values")
savefig(p2, "singular_values_plot.png")


U_k=truncated_svd_result.U

kmeans_result = kmeans(transpose(U_k), config.num_clusters)

# Extract the cluster assignments for each user
cluster_assignments = kmeans_result.assignments

# Print the cluster centers
println("Cluster Centers:")
println(kmeans_result.centers)
p3=scatter(U_k[:, 1], U_k[:, 2], group=cluster_assignments, xlabel="Feature 1", ylabel="Feature 2", title="User Clusters")
savefig(p3, "user_clusters.png")


#cluster_movie_forecasts=transpose(kmeans_result.centers)*S_k*transpose(V_k)
println("kmeans_result.centers: $(kmeans_result.centers)")
cluster_movie_forecasts=predict_from_users(transpose(kmeans_result.centers), truncated_svd_result)
println("Cluster movie forecasts: $(size(cluster_movie_forecasts))")
#num_movies= size(cluster_movie_forecasts, 2)
#sorted_cluster_ratings= [sortperm(cluster_movie_forecasts[i, :], rev=true) for i in 1:num_clusters]

#queues= create_queues(num_clusters, num_movies, sorted_cluster_ratings)
#println("Cluster 1 top 10 movies: ", [dequeue!(queues[1]) for _ in 1:10])


Starting up...
Loaded 32000204 ratings
Loaded 3621 users
U: (2896, 1489), S: (1489,), V: (1489, 1489)
Truncated SVD to 4 dimensions
Split 2896 train users and 725 test users
train_users: [172322, 164721, 154634, 188936, 8705, 65784, 129909, 141034, 62623, 94459, 173188, 188121, 171454, 131420, 133878, 179914, 38631, 99265, 118837, 114165, 197704, 144987, 135234, 148559, 44305, 143401, 71078, 106932, 21713, 78195, 159164, 91696, 157163, 173914, 129726, 75772, 95719, 111091, 164873, 167593, 170502, 9238, 181118, 168578, 139018, 982, 150917, 145379, 187247, 29008, 185306, 65925, 123678, 136641, 101806, 188545, 42572, 162854, 157595, 56494, 189665, 37721, 153876, 15279, 11862, 52255, 85529, 75880, 172034, 129188, 24575, 171519, 48436, 183120, 185637, 19465, 17869, 168454, 11453, 141567, 108595, 108965, 61651, 36944, 105063, 169909, 149737, 124558, 127189, 46351, 159776, 42317, 162993, 59051, 60059, 88964, 116927, 122061, 121845, 109263, 174566, 186539, 197527, 27591, 188533, 50693, 36370, 

In [70]:
cluster_movie_forecasts

7×1489 Matrix{Float64}:
  0.0410297   0.0083931   -0.0798117  …  -0.400587  -0.200408  0.111531
  0.0213742   0.0564513   -0.260315      -0.651809  -0.783684  0.250918
 -0.136482   -0.00394437  -0.122915      -0.216545  -0.166057  0.0147421
 -0.345926    0.00482591  -0.297021      -0.343644  -0.469478  0.0206118
 -0.669646    0.0170148   -0.254425      -0.179854  -0.133091  0.00722013
 -0.181826    0.0631327   -0.126484   …  -0.3329    -0.314324  0.208107
  0.186493   -0.020702    -0.225013      -0.59177   -0.609906  0.0599894

In [ ]:
#env=Environment(1,da
using Distributions
mutable struct Environment
    available_movies
    movie_reviews
end

#available_movies=dataset_test.raw_matrix[user_ind, :]
#available_movies = findall(!ismissing, dataset_test.raw_matrix[user_ind, :])
#env=Environment(available_movies,dataset_test.raw_matrix)
reward(env,user,movie)=env.movie_reviews[user,movie]

new_env(env,movie)=Environment(setdiff(env.available_movies, Set([movie])),env.movie_reviews)
#env=new_env(env,1)

policy2(env::Environment, user::Int, movie::Int) = rand(env.available_movies)
#policy(env,1,1)

struct DirichletPolicy #<: Policy 
    counts
    cluster_centers
    cluster_forecasts
end

function (π::DirichletPolicy)(s::Environment, user::Int, movie::Int)
    alpha=ones(length(π.counts))
    dirichlet_prior= Dirichlet(alpha+π.counts)
    cluster_sample= rand(dirichlet_prior)
    cluster_dist= Categorical(cluster_sample)
    cluster= rand(cluster_dist)

    forecasts=π.cluster_forecasts[cluster, s.available_movies]
    #println("forecasts: $(forecasts)")

    movie_pick= argmax(forecasts)
    #println("movie_pick: $(movie_pick)")
    return (cluster,s.available_movies[movie_pick])
end

function updated_policy(π::DirichletPolicy, user_ind, movie_ind, actual_rating)
    #average_user_rating=mean(dataset_test.raw_matrix[user_ind, :])
    average_user_rating=mean(skipmissing(dataset_test.raw_matrix[user_ind, :]))
    movie_forecasts=π.cluster_forecasts[:, movie_ind].+average_user_rating*0
    best_forecast=argmin((movie_forecasts.-actual_rating).^2)
    π.counts[best_forecast]+=1
    #println("updated counts: $(π.counts)")
    return π
end

function forecast_error(π::DirichletPolicy, user_ind, movie_ind, actual_rating,a)
    #average_user_rating=mean(dataset_test.raw_matrix[user_ind, :])
    average_user_rating=mean(skipmissing(dataset_test.raw_matrix[user_ind, :]))
    movie_forecasts=π.cluster_forecasts[:, movie_ind].+average_user_rating*0
    best_forecast=argmin((movie_forecasts.-actual_rating).^2)
    π.counts[best_forecast]+=1
    #println("updated counts: $(π.counts)")
    return (movie_forecasts[a]-actual_rating)^2
end


#π=DirichletPolicy(ones(7),kmeans_result.centers,cluster_movie_forecasts)
#π=policy2(env,1,1)
#policy2(env::Environment, user::Int, movie::Int) = rand(env.available_movies)

function simulate_episode(env, user_ind, policy,num_steps)
    all_tracking=[]
    for i in 1:num_steps
        tracking=Dict{String,Any}()
        (cluster,a)=policy(env,user_ind,1)
        print("policy: $(a)")
        r=reward(env,user_ind,a)
        env=new_env(env,a)
        #println("a=$(a),r=$(r)")
        tracking["i"]=i
        tracking["user"]=1
        tracking["movie"]=a
        tracking["reward"]=r
        tracking["action"]=a
        #tracking["error"]=forecast_error(policy,user_ind,r,a)

        tracking["policy_counts"]=copy(policy.counts)
        policy=updated_policy(policy,user_ind,a,r)
        #println("updated policy: $(policy.counts)")
        #tracking["state"]=env
        all_tracking=push!(all_tracking,tracking)
    end
    println("user_ind=$(user_ind)")
    return all_tracking
end
#=
all_user_tracking=[]
for user_ind in 1:length(dataset_test.user_map)
    available_movies = findall(!ismissing, dataset_test.raw_matrix[user_ind, :])
    env=Environment(available_movies,dataset_test.imputed_matrix)
    π=DirichletPolicy(ones(7),kmeans_result.centers,cluster_movie_forecasts)
    #π=policy2
    user_tracking=simulate_episode(env,user_ind,π,100)
    user_tracking_processed=DataFrame(user_tracking)
    all_user_tracking=push!(all_user_tracking,user_tracking_processed)
end
=#

function run_all_users(dataset_test, kmeans_result, cluster_movie_forecasts, num_clusters, num_steps=100)
    all_user_tracking=[]
    for user_ind in 1:length(dataset_test.user_map)
        available_movies = findall(!ismissing, dataset_test.raw_matrix[user_ind, :])
        env=Environment(available_movies,dataset_test.imputed_matrix)
        π=DirichletPolicy(ones(num_clusters),kmeans_result.centers,cluster_movie_forecasts)
        #π=policy2
        user_tracking=simulate_episode(env,user_ind,π,num_steps)
        user_tracking_processed=DataFrame(user_tracking)
        all_user_tracking=push!(all_user_tracking,user_tracking_processed)
    end
    return all_user_tracking
end

all_user_tracking=run_all_users(dataset_test, kmeans_result, cluster_movie_forecasts,config.num_clusters,100)
#all_tracking=simulate_episode(env,1,policy)

#tracking_matrices=Dict()

#averages=Dict()
#for field in ["reward"]
#    averages[field]=mean([x[field] for x in all_tracking])
#end
#=
#simulate_episode
#keep track of 
#1. user
#2. movie
#3. reward
#4. action
#5. state
#6. counts
#7. predicted rating
#8. actual rating
all_tracking=[]
for i in 1:10
    tracking=Dict{String,Any}()
    a=policy(env,user_ind,1)
    r=reward(env,user_ind,a)
    env=new_env(env,a)
    println("a=$(a),r=$(r)")
    tracking["i"]=i
    tracking["user"]=1
    tracking["movie"]=a
    tracking["reward"]=r
    tracking["action"]=a
    #tracking["state"]=env
    all_tracking=push!(all_tracking,tracking)
end
all_tracking
all_rewards=[x["reward"] for x in all_tracking]

=#
#env=Environment(Set(1:20),rand(1:5,num_users,num_movies))

725-element Vector{Any}:
 100×6 DataFrame
 Row │ reward     movie  action  policy_counts                      user   i   ⋯
     │ Float64    Int64  Int64   Array…                             Int64  Int ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │  1.81263     923     923  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1…      1      ⋯
   2 │  1.31263     346     346  [1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1…      1
   3 │  1.31263     786     786  [1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1…      1
   4 │  0.812634   1329    1329  [1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 1…      1
   5 │  1.81263     247     247  [1.0, 2.0, 2.0, 1.0, 2.0, 1.0, 2…      1      ⋯
   6 │  1.31263     756     756  [1.0, 2.0, 2.0, 1.0, 3.0, 1.0, 2…      1
   7 │  1.81263    1396    1396  [1.0, 2.0, 2.0, 1.0, 4.0, 1.0, 2…      1
   8 │  0.812634    462     462  [1.0, 2.0, 2.0, 1.0, 5.0, 1.0, 2…      1
  ⋮  │     ⋮        ⋮      ⋮                     ⋮                    ⋮      ⋮ ⋱
  94 │  1.81263    1294    1

In [13]:
config.k

4

In [15]:
#all_user_tracking[1][!,"reward"]
sum_tracking=mean([x[!,"reward"] for x in all_user_tracking])
all_user_tracking[1]
cumsum(sum_tracking)

#all_user_tracking[10][!,"policy_counts"]   

100-element Vector{Float64}:
  0.7951731333486474
  1.6337945425593634
  2.3462090552183557
  3.1262097747739
  3.8434518736397885
  4.604142248367747
  5.27655676102674
  5.93035058403056
  6.618627165655068
  7.23104167831406
  ⋮
 48.56834206117901
 48.93248071176904
 49.27524005201424
 49.62558559915599
 50.03593114629774
 50.398000831370524
 50.75593258540883
 51.08282985668851
 51.46627885210612

In [ ]:
dataset_test.raw_matrix[user_ind, :]

RatingsData(Union{Missing, Float64}[0.5 3.0 … missing missing; 2.5 missing … 2.5 missing; … ; 4.0 missing … missing missing; 4.0 missing … missing missing], [-1.9556313993174061 0.5443686006825939 … 0.0 0.0; -1.0524475524475525 0.0 … -1.0524475524475525 0.0; … ; 0.37070938215102966 0.0 … 0.0 0.0; 0.24539877300613488 0.0 … 0.0 0.0], Dict(95987 => 2, 36757 => 360, 29843 => 125, 126467 => 237, 180259 => 126, 4511 => 361, 161120 => 238, 96694 => 239, 58093 => 3, 104647 => 127…), Dict(77561 => 1, 2108 => 2, 4700 => 3, 1175 => 4, 1953 => 5, 719 => 6, 2288 => 7, 4901 => 8, 93510 => 9, 1028 => 10…), [2.455631399317406, 3.5524475524475525, 4.042239685658153, 3.6073788546255505, 3.9270205066344994, 3.6766826923076925, 3.305956678700361, 3.797507788161994, 3.358707557502738, 3.6  …  3.4267990074441688, 3.168261562998405, 3.5277098614506928, 3.562814070351759, 3.4171122994652405, 3.6262699564586356, 3.7325, 3.878887070376432, 3.6292906178489703, 3.754601226993865])